In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [9]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})
path = "/Users/joejohns/Downloads/data_loans.csv"


In [10]:

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


Loan_ID Gender Married Dependents Education Self_Employed  \
0  LP001002   Male      No          0  Graduate            No   
1  LP001003   Male     Yes          1  Graduate            No   
2  LP001005   Male     Yes          0  Graduate           Yes   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0         Urban           Y

## data processing from round 1


In [11]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [12]:


for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [13]:

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [14]:


for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i
    
#impute_most_freq = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#data["Dependents"] = impute_most_freq.fit_transform(data["Dependents"])

#gah f-off with your 2D array non-sense ... I'll write my own function


In [15]:

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [16]:
data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [17]:
## These are the two numerical features with NaN .. here we set up two new categories to capture these NaN since
## they seem decently predictive. ... Now what to do with the NaN values? I guess just do mean and most-common


data['Loan_Amount_Term_unk'] = 'No' #set default to 'No' then set a few to be 'Yes'
data['Loan_Amount_unk'] = 'No'  
data.loc[(data['Loan_Amount_Term'].isnull()) , 'Loan_Amount_Term_unk'] = 'Yes'
data.loc[(data['Loan_Amount'].isnull()) , 'Loan_Amount_unk'] = 'Yes'


In [18]:

cat_nom_feats = ['Gender', 'Married','Self_Employed',  'Credit_History', 'Property_Area',
        'Loan_Amount_Term_unk', 'Loan_Amount_unk' ]

#we don't want to do this for cat_ord_feats, because those will be numeric; we do impute_most_freq instead
for cat in cat_nom_feats:
    if data[cat].isnull().sum()>0: 
        data.loc[data[cat].isnull(), cat] = 'unk'

#### Done with cleaning categorical features ... 
note: we have made preliminary decision to make all NaN in categorical columns into new cat_val ... \
we'll see how this works out ... in some cases it seems to be a good predictor.

In phase 2 investigation, below, we will investigate the naive predictive strength of all the \
categorical features (including NaN as 'unk')


In [19]:
##baseline models
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.neighbors import KNeighborsClassifier # The k-nearest neighbor classifier
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV # For optimization


I should play around with each step (eg onehot) and make sure it works ...\
Ideally I should also play around with ColumnTransformer (make a one step pipeline and see the result ?)

In [20]:
from sklearn.preprocessing import OneHotEncoder
#One can discard categories not seen during fit:

enc = OneHotEncoder(handle_unknown='ignore')

dummies = pd.get_dummies()


In [21]:
data2 = data.head(5)

In [24]:
enc.fit(data2.loc[:, cat_nom_feats])

OneHotEncoder(handle_unknown='ignore')

In [25]:
enc.categories_

[array(['Male'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Yes'], dtype=object),
 array(['Rural', 'Urban'], dtype=object),
 array(['No'], dtype=object),
 array(['No', 'Yes'], dtype=object)]

In [26]:
enc.transform(data2.loc[:, cat_nom_feats])

<5x11 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [27]:
enc.transform(data2.loc[:, cat_nom_feats]).toarray()

array([[1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.],
       [1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.],
       [1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0.],
       [1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0.],
       [1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.]])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
#One can discard categories not seen during fit:

enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
enc.categories_
[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
enc.transform([['Female', 1], ['Male', 4]]).toarray()
array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])
enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])
array([['Male', 1],
       [None, 2]], dtype=object)
enc.get_feature_names(['gender', 'group'])
array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
  dtype=object)

In [4]:
from sklearn.preprocessing import OneHotEncoder
#One can discard categories not seen during fit:

enc = OneHotEncoder(handle_unknown='ignore')

In [2]:
X = [['Male', 1], ['Female', 3], ['Female', 2]]
X

[['Male', 1], ['Female', 3], ['Female', 2]]

In [5]:
enc.fit(X)

OneHotEncoder(handle_unknown='ignore')

In [6]:
enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [8]:
enc.transform([['Female', 1], ['Male', 4]]).toarray()

array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [ ]:
enc.fit(data)